In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()
X_train, X_test = X_train/255, X_test/255

In [3]:
X = X_input = keras.layers.Input(X_train.shape[1:])
X = keras.layers.BatchNormalization()(X)
X = X_skip = keras.layers.Conv2D(16, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Conv2D(16, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.Add()([X, X_skip])
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = X_skip = keras.layers.Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.Add()([X, X_skip])
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = X_skip = keras.layers.Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.Add()([X, X_skip])
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Conv2D(128, (3,3), padding='valid', kernel_initializer='he_normal')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(np.max(Y_train)+1, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

### search for best cosine annealing max learning rate

In [4]:
hist = M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=10, callbacks=[
    keras.callbacks.LearningRateScheduler(lambda epoch,lr: 0.5*((epoch+1)/10), verbose=1),
    keras.callbacks.EarlyStopping(monitor='loss', patience=2, verbose=1),
])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 0.05.
50000/50000 [==============================] - 57s 1ms/step - loss: 1.5459 - acc: 0.4381 - val_loss: 1.9256 - val_acc: 0.4286
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 0.1.
50000/50000 [==============================] - 54s 1ms/step - loss: 1.0897 - acc: 0.6186 - val_loss: 1.8453 - val_acc: 0.5414
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 0.15.
50000/50000 [==============================] - 53s 1ms/step - loss: 0.9723 - acc: 0.6670 - val_loss: 1.4169 - val_acc: 0.5868
Epoch 4/10

Epoch 00004: LearningRateScheduler setting learning rate to 0.2.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.9307 - acc: 0.6879 - val_loss: 1.5187 - val_acc: 0.6040
Epoch 5/10

Epoch 00005: LearningRateScheduler setting learning rate to 0.25.
50000/50000 [==============================] - 54s 1ms/ste

In [5]:
max_lr = 0.5*((np.argmin(hist.history['loss'])+1)/10)
max_lr

0.45

### cosine annealing

In [6]:
def cosine_annealing(i, lr_min, lr_max):
    i = i + np.power(2,2)
    log_i = np.log2(i)
    t_min,t_max = np.power(2,np.floor(log_i)), np.power(2,np.floor(log_i)+1)-1
    return lr_min + np.cos(((i-t_min)/(t_max-t_min))*(np.pi/2))*(lr_max-lr_min)
M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=60, callbacks=[
    keras.callbacks.LearningRateScheduler(
        lambda epoch,lr: cosine_annealing(epoch,max_lr*0.25,max_lr), verbose=1)
])

Train on 50000 samples, validate on 10000 samples
Epoch 1/60

Epoch 00001: LearningRateScheduler setting learning rate to 0.45.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.7281 - acc: 0.7688 - val_loss: 0.9205 - val_acc: 0.7283
Epoch 2/60

Epoch 00002: LearningRateScheduler setting learning rate to 0.40478357377724805.
50000/50000 [==============================] - 53s 1ms/step - loss: 0.7141 - acc: 0.7795 - val_loss: 0.7832 - val_acc: 0.7557
Epoch 3/60

Epoch 00003: LearningRateScheduler setting learning rate to 0.28125000000000006.
50000/50000 [==============================] - 53s 1ms/step - loss: 0.5420 - acc: 0.8247 - val_loss: 0.9227 - val_acc: 0.7191
Epoch 4/60

Epoch 00004: LearningRateScheduler setting learning rate to 0.11250000000000002.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.3892 - acc: 0.8707 - val_loss: 0.6410 - val_acc: 0.8035
Epoch 5/60

Epoch 00005: LearningRateScheduler setting learning rate to 0.45.
50000/50000

50000/50000 [==============================] - 54s 1ms/step - loss: 0.3714 - acc: 0.8874 - val_loss: 1.2731 - val_acc: 0.7366
Epoch 39/60

Epoch 00039: LearningRateScheduler setting learning rate to 0.40759198294879645.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.3531 - acc: 0.8924 - val_loss: 1.5498 - val_acc: 0.6797
Epoch 40/60

Epoch 00040: LearningRateScheduler setting learning rate to 0.39891743690447845.
50000/50000 [==============================] - 55s 1ms/step - loss: 0.3358 - acc: 0.8994 - val_loss: 1.2462 - val_acc: 0.7519
Epoch 41/60

Epoch 00041: LearningRateScheduler setting learning rate to 0.38950766140745574.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.3240 - acc: 0.9034 - val_loss: 1.4744 - val_acc: 0.7134
Epoch 42/60

Epoch 00042: LearningRateScheduler setting learning rate to 0.3793868112164733.
50000/50000 [==============================] - 54s 1ms/step - loss: 0.3100 - acc: 0.9060 - val_loss: 1.1497 - val_acc: 0.